# How good at this am I?

We are going to be testing the quality of these models, and we'll start with the Brier score.

https://en.wikipedia.org/wiki/Brier_score

## A note on buckets

Until I figure out how the buckets are created, I'm just going to stick with the average of the past 7 weeks, treat that as the middle bucket and then go from there. I got 173, so the buckets look like this:

139 or fewer
140 - 149
150 - 159
160 - 169
170 - 179
etc

In [103]:
import sys
import datetime

import pandas as pd
import numpy as np

sys.path.append("..")

import src.utils as utils
import src.poisson as poisson

In [104]:
df = utils.tweets('realDonaldTrump')
by_day = df.groupby(df.index.date).count()
by_day.index = pd.to_datetime(by_day.index)

## Timing

We'll start 30 days ago, and use at most 90 days

In [105]:
start_date = datetime.datetime.now() - datetime.timedelta(days=30)
end_of_data = start_date - datetime.timedelta(days=90)

## rolling sums

In [106]:
for i in range(1,8):
    by_day[f"Sum_{i}"] = by_day["ID"].shift(1).rolling(i).sum()

In [107]:
by_day

,ID,Sum_1,Sum_2,Sum_3,Sum_4,Sum_5,Sum_6,Sum_7
2017-01-01,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-02,7,6.0,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-03,10,7.0,13.0,NaN,NaN,NaN,NaN,NaN
2017-01-04,10,10.0,17.0,23.0,NaN,NaN,NaN,NaN
2017-01-05,6,10.0,20.0,27.0,33.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2020-05-02,40,38.0,101.0,122.0,144.0,182.0,199.0,226.0
2020-05-03,30,40.0,78.0,141.0,162.0,184.0,222.0,239.0
2020-05-04,11,30.0,70.0,108.0,171.0,192.0,214.0,252.0
2020-05-05,32,11.0,41.0,81.0,119.0,182.0,203.0,225.0


In [108]:
by_day.sort_index(ascending=False, inplace=True)

In [109]:
to_test = by_day[start_date:end_of_data]
to_test

,ID,Sum_1,Sum_2,Sum_3,Sum_4,Sum_5,Sum_6,Sum_7
2020-04-07,19,20.0,58.0,111.0,126.0,141.0,148.0,172.0
2020-04-06,20,38.0,91.0,106.0,121.0,128.0,152.0,167.0
2020-04-05,38,53.0,68.0,83.0,90.0,114.0,129.0,168.0
2020-04-04,53,15.0,30.0,37.0,61.0,76.0,115.0,147.0
2020-04-03,15,15.0,22.0,46.0,61.0,100.0,132.0,154.0
...,...,...,...,...,...,...,...,...
2020-01-13,25,31.0,47.0,86.0,115.0,117.0,150.0,158.0
2020-01-12,31,16.0,55.0,84.0,86.0,119.0,127.0,130.0
2020-01-11,16,39.0,68.0,70.0,103.0,111.0,114.0,121.0
2020-01-10,39,29.0,31.0,64.0,72.0,75.0,82.0,114.0


## Make model

In [110]:
days_left = 7
tweets_so_far = 0
lower = 139

predict = poisson.basic_poisson(by_day[start_date:end_of_data]["ID"], days_left, tweets_so_far, lower)

In [111]:
predict

{'139': 0.0,
 '140-149': 0.0,
 '150-159': 0.0,
 '160-169': 0.0,
 '170-179': 0.1,
 '180-189': 0.8,
 '190-199': 3.8,
 '200-209': 11.0,
 '210': 81.6}

In [112]:
prediction_date = start_date + datetime.timedelta(days=8)

In [113]:
prediction_date.date()

datetime.date(2020, 4, 15)

In [114]:
lower = 139

def which_bucket(lower, value, buckets=9, gap=10):
    
    outcomes = dict()
    
    for i in range(buckets):
        if i == 0:
            if value < lower:
                outcomes[str(lower)] = 1
            else:
                outcomes[str(lower)] = 0
        elif i == buckets - 1:
            lower_bound = (lower + i * gap) - (gap - 1)
            if value > lower_bound:
                outcomes[str(lower_bound)] = 1
            else:
                outcomes[str(lower_bound)] = 0
                
        else:
            lower_bound = (lower + i * gap) - (gap - 1)
            upper_bound = (lower + i * gap)
                
            label = f"{lower_bound}-{upper_bound}"
                
            if value >= lower_bound and value <= upper_bound:
                outcomes[label] = 1
            else:
                outcomes[label] = 0
                    
                
                
    return outcomes
                
which_bucket(lower, 201)

{'139': 0,
 '140-149': 0,
 '150-159': 0,
 '160-169': 0,
 '170-179': 0,
 '180-189': 0,
 '190-199': 0,
 '200-209': 1,
 '210': 0}

In [115]:
outcome = by_day.at[prediction_date.strftime("%Y-%-m-%d"), "Sum_7"][0]

In [116]:
outcome_mask = which_bucket(lower, outcome)

In [117]:
def brier_score(predictions, ground_truth):
    
    # triple check the buckets have the same keys
    if not predictions.keys() == ground_truth.keys():
        print("Dict keys are not the same")
        return False
    sum = 0
    for i, key in enumerate(predictions):
        x = predictions[key] * .001 - ground_truth[key]
        sum += x ** 2
    
    return sum / (i + 1)
    
brier_score(predict, outcome_mask)

0.10942162777777778

## And for 6 days out

In [121]:
# get the tweets
tweets_date = start_date + datetime.timedelta(days=1)
tweets_so_far = by_day.at[tweets_date.strftime("%Y-%-m-%d"), "Sum_1"][0]
days_left = 6

predict = poisson.basic_poisson(by_day[start_date:end_of_data]["ID"], days_left, tweets_so_far, lower)

In [123]:
brier_score(predict, outcome_mask)

0.1062408477777778

## Okay let's come up with a week

In [128]:

# we already have the outcome, but it would be computed here
scores = list()

for i in range(7):
    if i == 0:
        # a week away
        predict = poisson.basic_poisson(by_day[start_date:end_of_data]["ID"], 0, 0, lower)
        
        scores.append(brier_score(predict, outcome_mask))
    else:
        tweets_date = start_date + datetime.timedelta(days=i)
        tweets_so_far = by_day.at[tweets_date.strftime("%Y-%-m-%d"), f"Sum_{i}"][0]
        print(tweets_so_far)
        
        predict = poisson.basic_poisson(by_day[start_date:end_of_data]["ID"],
                                        days_left,
                                        tweets_so_far,
                                        lower)
        print(predict)
        scores.append(brier_score(predict, outcome_mask))
                                  
        
scores

19.0
{'139': 0.0, '140-149': 0.0, '150-159': 0.0, '160-169': 0.1, '170-179': 0.8999999999999999, '180-189': 4.6, '190-199': 13.700000000000001, '200-209': 23.599999999999998, '210': 51.0}
41.0
{'139': 0.0, '140-149': 0.0, '150-159': 0.0, '160-169': 0.0, '170-179': 0.0, '180-189': 0.1, '190-199': 0.6, '200-209': 3.5000000000000004, '210': 95.0}
49.0
{'139': 0.0, '140-149': 0.0, '150-159': 0.0, '160-169': 0.0, '170-179': 0.0, '180-189': 0.0, '190-199': 0.1, '200-209': 0.8999999999999999, '210': 98.8}
129.0
{'139': 0.0, '140-149': 0.0, '150-159': 0.0, '160-169': 0.0, '170-179': 0.0, '180-189': 0.0, '190-199': 0.0, '200-209': 0.0, '210': 100.0}
148.0
{'139': 0.0, '140-149': 0.0, '150-159': 0.0, '160-169': 0.0, '170-179': 0.0, '180-189': 0.0, '190-199': 0.0, '200-209': 0.0, '210': 100.0}
188.0
{'139': 0.0, '140-149': 0.0, '150-159': 0.0, '160-169': 0.0, '170-179': 0.0, '180-189': 0.0, '190-199': 0.0, '200-209': 0.0, '210': 100.0}


[0.11222222222222222,
 0.1062408477777778,
 0.11133751333333335,
 0.11199580666666667,
 0.11222222222222222,
 0.11222222222222222,
 0.11222222222222222]

## Okay those results look suss, see how it gives 210 a 100% 